# Tricks of the trade TF/Keras [small dataset]

In this script we build a small multilayer perceptron with two hidden layers having 500 and 50 neurons each for classifying the MNIST database of handwritten digits using Keras. It uses a small data set so that it can be trained on the CPU.

Below are several experiments.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import numpy as np

import time
import tensorflow as tf
tf.set_random_seed(1)

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras
import sys
keras.__version__, tf.__version__, sys.version_info

Using TensorFlow backend.


('1.2.2',
 '1.0.0',
 sys.version_info(major=3, minor=4, micro=3, releaselevel='final', serial=0))

In [2]:
# To be compatible with python3 and python2
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gzip

with gzip.open('../../lasagne/mnist_4000.pkl.gz', 'rb') as f:
    if sys.version_info.major > 2:
        (X,y) = pickle.load(f, encoding='latin1')
    else:
        (X,y) = pickle.load(f)
PIXELS = len(X[0,0,0,:])

print(X.shape, y.shape, PIXELS) #As read
# We need to reshape for the MLP
X = X.reshape([4000, 784])
np.shape(X)

(4000, 1, 28, 28) (4000,) 28


(4000, 784)

In [7]:
# Taken from http://stackoverflow.com/questions/29831489/numpy-1-hot-array
def convertToOneHot(vector, num_classes=None):
    result = np.zeros((len(vector), num_classes), dtype='int32')
    result[np.arange(len(vector)), vector] = 1
    return result

### Suggestions for the experiment

Let the experiments run for 100 epochs. You might need to restart the kernel so that namings of the layers are the same

* with init zero 
* with sigmoid activation 
* with ReLU activation
* with dropout (p=0.3)
* with batch-normalization and dropout

In [4]:
### First model with all zeros
name = 'sigmoid_init0'
model = Sequential()
model.add(Dense(500, batch_input_shape=(None, 784), init='zero'))
model.add(Activation('sigmoid'))

model.add(Dense(50,init='zero'))
model.add(Activation('sigmoid'))

model.add(Dense(10, activation='softmax',init='zero'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [13]:
### Model with default initialization 
name = 'sigmoid'
model = Sequential()
model.add(Dense(500, batch_input_shape=(None, 784)))
#model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('sigmoid'))

model.add(Dense(50))
#model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('sigmoid'))

model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [4]:
### Model with default initialization 
name = 'relu'
model = Sequential()
model.add(Dense(500, batch_input_shape=(None, 784)))
#model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(50))
#model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [4]:
### Model with default initialization 
name = 'dropout'
model = Sequential()
model.add(Dense(500, batch_input_shape=(None, 784)))
model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [3]:
### Model with default initialization 
name = 'dropout_batch'
model = Sequential()
model.add(Dense(500, batch_input_shape=(None, 784)))
model.add(Dropout(0.3))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Dropout(0.3))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 500)           392500      dense_input_1[0][0]              
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 500)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 500)           2000        dropout_1[0][0]                  
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 500)           0           batchnormalization_1[0][0]       
___________________________________________________________________________________________

### Untrained model


In [5]:
np.log(0.1)

-2.3025850929940455

In [8]:
model.evaluate(X[0:2000], convertToOneHot(y[0:2000],10))

1984/2000 [============================>.] - ETA: 0s

[2.6063219032287597, 0.082000000000000003]

In [11]:
model.predict_classes(X[0:10])

10/10 [==============================] - 0s


array([9, 4, 0, 9, 7, 9, 9, 9, 9, 9])

In [16]:
#model.save('/tmp/start.keras')
#%ls -hl /tmp/start.keras

## Training

In [13]:
log_dir='/notebooks/tensorflow/path_to_fc_nets/tb/' + name

In [14]:
tensorboard = keras.callbacks.TensorBoard(
    log_dir='/notebooks/tensorflow/path_to_fc_nets/tb/' + name + '/', 
    write_graph=True,
    histogram_freq=1
)
history = model.fit(X[0:2400], 
          convertToOneHot(y[0:2400],10), 
          nb_epoch=100, 
          batch_size=128, 
          callbacks=[tensorboard],
          validation_data=[X[2400:3000], convertToOneHot(y[2400:3000],10)], verbose=2)

# Attention, if you run this in a docker container, setting verbose=1 sometimes kills the whole container 
# ERRO[0695] error getting events from daemon: EOF

# Sometimes also the TensorBoard callback kills the container

Train on 2400 samples, validate on 600 samples
INFO:tensorflow:Summary name dense_1_W:0 is illegal; using dense_1_W_0 instead.
INFO:tensorflow:Summary name dense_1_b:0 is illegal; using dense_1_b_0 instead.
INFO:tensorflow:Summary name batchnormalization_1_gamma:0 is illegal; using batchnormalization_1_gamma_0 instead.
INFO:tensorflow:Summary name batchnormalization_1_beta:0 is illegal; using batchnormalization_1_beta_0 instead.
INFO:tensorflow:Summary name batchnormalization_1_running_mean:0 is illegal; using batchnormalization_1_running_mean_0 instead.
INFO:tensorflow:Summary name batchnormalization_1_running_std:0 is illegal; using batchnormalization_1_running_std_0 instead.
INFO:tensorflow:Summary name dense_2_W:0 is illegal; using dense_2_W_0 instead.
INFO:tensorflow:Summary name dense_2_b:0 is illegal; using dense_2_b_0 instead.
INFO:tensorflow:Summary name batchnormalization_2_gamma:0 is illegal; using batchnormalization_2_gamma_0 instead.
INFO:tensorflow:Summary name batchnorma